In [1]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

%run 1.tokenizer.ipynb

tokenizer

In [2]:
%run 2.dataset.ipynb

dataset = Dataset(negative_label=False, with_answer=False)

len(dataset), dataset[0]

(1000000, {'text': '4714+3261=', 'label': 1})

In [3]:
%run 3.model.ipynb

model_actor = torch.load('model/ppo', weights_only=False).to(device)

In [4]:
#====question====
question = random.choices(dataset, k=16)
question = [i['text'] for i in question]

question = tokenizer(question,
                     device=device,
                     add_bos_token=True,
                     add_eos_token=False,
                     padding_side='left')['input_ids']

#====gen====
gen = generate(model_actor,
               input_ids=question,
               pad_token_id=tokenizer.pad_token_id,
               eos_token_id=tokenizer.eos_token_id)
gen = gen[:, question.shape[1]:]

for q, g in zip(question, gen):
    q = tokenizer.decode(q)
    g = tokenizer.decode(g)
    a = q
    a = a[a.index(tokenizer.bos_token) + 1:]
    a = a[:a.index('=')]
    a = int(eval(a))

    print({'question': q, 'answer': a, 'gen': g})

{'question': 'B9199+1350=', 'answer': 10549, 'gen': '10549E'}
{'question': 'B999+8643=', 'answer': 9642, 'gen': '9642E'}
{'question': 'B2302+1625=', 'answer': 3927, 'gen': '3927E'}
{'question': 'B8277+3408=', 'answer': 11685, 'gen': '11685E'}
{'question': 'B939+3877=', 'answer': 4816, 'gen': '4316E'}
{'question': 'B8709+738=', 'answer': 9447, 'gen': '9447E'}
{'question': 'B8602+3038=', 'answer': 11640, 'gen': '11640E'}
{'question': 'B9418+6823=', 'answer': 16241, 'gen': '16241E'}
{'question': 'B7894+1129=', 'answer': 9023, 'gen': '9023E'}
{'question': 'B7220+2682=', 'answer': 9902, 'gen': '9902E'}
{'question': 'B4999+1116=', 'answer': 6115, 'gen': '6115E'}
{'question': 'B2613+8340=', 'answer': 10953, 'gen': '10953E'}
{'question': 'B7482+5164=', 'answer': 12646, 'gen': '12646E'}
{'question': 'B2012+4254=', 'answer': 6266, 'gen': '6266E'}
{'question': 'B4528+4072=', 'answer': 8600, 'gen': '8600E'}
{'question': 'B8456+75=', 'answer': 8531, 'gen': '8521E'}
